In [4]:
import numpy as np
import pandas as pd
from operator import itemgetter

In [5]:
y_at = pd.DataFrame({"Outlook" :["Sunny","Sunny","Overcast","Rain","Rain","Rain","Overcast","Sunny","Sunny","Rain","Sunny","Overcast","Overcast","Rain"],"Temperature":["Hot","Hot","Hot","Mild","Cool","Cool","Cool","Mild","Cool","Mild","Mild","Mild","Hot","Mild"],"Humidity":["High","High","High","High","Normal", "Normal","Normal","High","Normal","Normal","Normal","High","Normal","High"],"Wind": ["Weak", "Strong", "Weak", "Weak", "Weak", "Strong","Strong", "Weak", "Weak", "Weak", "Strong","Strong", "Weak","Strong"]})

In [6]:
y = pd.DataFrame(["No", "No", "yes", "yes", "yes","No", "yes","No", "yes", "yes", "yes", "yes", "yes","No"])

In [8]:
def entropy(y):
    out = np.array(y)
    ent = 0
    for val in np.unique(out):
        temp = -(len(out[out == val])/len(y))*(np.log2(len(out[out == val])/len(y)))
        ent += temp
    return ent
        

In [9]:
def get_IG(y,att_y):
    att_y.columns = range(att_y.shape[1])
    temp_ig = 0
    
    for att in att_y[0].unique():
        
        temp_df = att_y[att_y[0] == att]
        H_x = entropy(temp_df[1])
        p_s = len(temp_df)/len(att_y)
        temp_ig += ((p_s*H_x))
        
    return entropy(y)-temp_ig

In [10]:
#test entropy calculation
entropy(y)

0.9402859586706311

In [11]:
def get_top_node(data,y):
    temp_df = pd.concat([data,y],axis = 1)
    temp_IGs = []
    for att in data.keys():
        single_df = temp_df[[att,y.keys()[0]]]
        IG = get_IG(y,single_df)
        temp_IGs.append((att, IG))
    
    return sorted(temp_IGs, key = itemgetter(1), reverse = True)[0]
    

In [12]:
get_top_node(y_at,pd.DataFrame(y))

('Outlook', 0.24674981977443933)

In [13]:
def set_branches(root,sub_tree,data, full_data):
    sub_tree[root] = {}
    for attr in full_data[root].unique():
            print("Working for attribute:",attr,"for node:",root)
            temp_data = data[data[root] == attr]
            temp_data_full = full_data[full_data[root] == attr]
            if len(temp_data_full[temp_data_full.columns[-1]].unique()) == 1:
                sub_tree[root][attr] =(temp_data_full[temp_data_full.columns[-1]].unique(),1)
                print("Tree created:", sub_tree)
                return sub_tree
            else:
                current_node = get_top_node(temp_data,temp_data_full[temp_data_full.columns[-1]])[0]
                print("Next node for working:",current_node)
                sub_tree[root][attr] = {}
                current_tree = sub_tree[root][attr]
                return set_branches(current_node,current_tree,temp_data,temp_data_full)


In [16]:
def fit_tree(data,target):
    full_data = pd.concat([data,target],axis = 1)
    tree = {}
    #get root node
    n = data.shape[0] - 1
    root = get_top_node(data,target)[0]
    print("Root Selected:", root)
    #get attributes of current node
    
    final_tree = set_branches(root,tree,data,full_data)
    '''
    for attr in full_data[root].unique():
        print("Working for attribute:",attr,"for node:",root)
        temp_data = data[data[root] == attr]
        temp_data_full = full_data[full_data[root] == attr]
        if len(temp_data_full[temp_data_full.columns[-1]].unique()) == 1:
            sub_tree[attr] =(temp_data_full[temp_data_full.columns[-1]].unique(),1)
            print("Tree created:", sub_tree)
        else:
            current_node = get_top_node(temp_data,temp_data_full[temp_data_full.columns[-1]])
            print("Next node for working:",current_node)
    '''
    
    return final_tree
    #check if a sub branch gives a defined answer
    
    #redo calculations for other subbranches
    

In [17]:
tree = fit_tree(y_at,y)

Root Selected: Outlook
Working for attribute: Sunny for node: Outlook
Next node for working: Humidity
Working for attribute: High for node: Humidity
Tree created: {'Humidity': {'High': (array(['No'], dtype=object), 1)}}


In [18]:
tree

{'Humidity': {'High': (array(['No'], dtype=object), 1)}}

In [52]:
get_IG(y,y_at)

Working for attribute: Weak
H_x = 0.8112781244591328
p_s = 0.5714285714285714
Working for attribute: Strong
H_x = 1.0
p_s = 0.42857142857142855


0.04812703040826949